In [61]:
from covid.simulator import Population
from covid.auxilliary import symptom_names
import numpy as np
import pandas as pd

from covid.policy import Policy

In [2]:
w = np.array([0.2, 0.1, 0.1, 0.1, 0.5, 0.2, 0.5, 1.0, 100])
assert w.shape[0] == 9, 'Shape of weights does not fit number of symptoms'

In [36]:
class Model:
    def __init__(self, nsymptom, nvacc):
        # Priors for the beta-bernoulli model
        self.a = np.ones(shape=[nvacc, nsymptom])/2 # using jeffreys prior
        self.b = np.ones(shape=[nvacc, nsymptom])/2 # using jeffreys prior
        self.nvacc = nvacc
        self.nsymptom = nsymptom

    def update(self, features, actions, outcomes):
        """
        for index in range(self.nvacc):
            print(outcomes[np.where(actions == (index-1))]) 
            print(actions)
            print(outcomes)
            if (np.sum(outcomes[np.where(actions == index - 1)], axis=1).size != 0):
                self.a[index] += np.sum(outcomes[np.where(actions == index - 1)], axis=1)
                self.b[index] += np.sum(outcomes[np.where(actions == index - 1)]==0, axis=1)\
                    - np.sum(outcomes[np.where(actions == index - 1)], axis=1)
            else: 
                self.b[index] += np.sum((outcomes==0)[np.where(actions == index - 1)], axis=1)
        """
        for index, outcome in enumerate(outcomes):
            self.a[int(actions[index])] += outcome[1:]
            self.b[int(actions[index])] += 1 - outcome[1:]

    def get_params(self):
        return self.a, self.b


    def get_prob(self, features, action):
        return self.a[action] / (self.a[action] + self.b[action])

    
    def retrain(self, features, actions, outcomes):
        # Use aggregated database of outcomes etc...

        self.a = np.ones(shape=[self.nvacc, self.nsymptom])/2 # using jeffreys prior
        self.b = np.ones(shape=[self.nvacc, self.nsymptom])/2 # using jeffreys prior

        self.update(features, actions, outcomes)

In [48]:
class Naive(Policy):
    def get_utility(self, features, action, outcome):
        """Here the utiliy is defined in terms of the outcomes obtained only, ignoring both the treatment and the previous condition.
        """
        actions = self.get_action(features)
        utility = 0
        utility -= 0.2 * sum(outcome[:,symptom_names['Covid-Positive']]) 
        utility -= 0.1 * sum(outcome[:,symptom_names['Taste']])
        utility -= 0.1 * sum(outcome[:,symptom_names['Fever']])
        utility -= 0.1 * sum(outcome[:,symptom_names['Headache']])
        utility -= 0.5 * sum(outcome[:,symptom_names['Pneumonia']])
        utility -= 0.2 * sum(outcome[:,symptom_names['Stomach']])
        utility -= 0.5 * sum(outcome[:,symptom_names['Myocarditis']])
        utility -= 1.0 * sum(outcome[:,symptom_names['Blood-Clots']])
        utility -= 100.0 * sum(outcome[:,symptom_names['Death']])
        return utility

    def set_model(self, model):
        self.model = model
    
    def get_action(self, features):
        """Get a completely random set of actions, but only one for each individual.

        If there is more than one individual, feature has dimensions t*x matrix, otherwise it is an x-size array.
        
        It assumes a finite set of actions.

        Returns:
        A t*|A| array of actions
        """
        n_obs = features.shape[0]
        actions = np.zeros(n_obs)
        #u_list = np.zeros((n_obs, self.n_actions))
        for index, t in enumerate(features):
            u_list = []
            for a in self.action_set:
                u_list.append(self.get_expected_utility(a, t))
            actions[index] = np.argmax(np.array(u_list))
        return actions

    def get_expected_utility(self, action, features):
        p = self.model.get_prob(features, action)
        return -np.dot(p, w)*(1+int(action != -1))

    def observe(self, features, action, outcomes):
        self.model.update(features, action, outcomes)
        

In [55]:
## Baseline simulator parameters
n_genes = 128
n_vaccines = 3
n_treatments = 4
n_population = 10000
n_symptoms = 9

# symptom names for easy reference
from covid.auxilliary import symptom_names

In [63]:
population = Population(n_genes, n_vaccines, n_treatments)
X = population.generate(n_population)
n_features = X.shape[1]

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [64]:
print("With a for loop")
vaccine_policy = Naive(2, np.array([-1, 0]))
vaccine_policy.set_model(Model(n_symptoms, 2))
# The simplest way to work is to go through every individual in the population
Y = np.zeros((n_population, n_symptoms+1))
A = np.zeros(n_population)
for t in range(n_population):
    #print("Person nr: ", t)
    a_t = vaccine_policy.get_action(X[t].reshape((1, n_features)))
    A[t] = a_t
    # Then you can obtain results for everybody
    y_t = population.vaccinate([t], a_t.reshape((1, 1)))
    Y[t] = y_t
    # Feed the results back in your policy. This allows you to fit the
    # statistical model you have.
    vaccine_policy.observe(X[t], a_t, y_t)

With a for loop
Initialising policy with  2 actions
A = { [-1  0] }


In [68]:
vaccine_policy.get_utility(X, A-1, Y)

-2000.0

In [50]:
a, b = vaccine_policy.model.get_params()
print(a[0]/(a[0] + b[0]))
print(a[1]/(a[1] + b[1]))

[9.99994987e-01 5.01268209e-06 5.01268209e-06 5.01268209e-06
 5.01268209e-06 5.01268209e-06 5.01268209e-06 5.01268209e-06
 5.01268209e-06]
[0.99803922 0.00196078 0.00196078 0.00196078 0.00196078 0.00196078
 0.00196078 0.00196078 0.00196078]
